In [1]:
#[Hipótese 1] Data por CNAE

In [2]:
# Maior movimentação, elencar o maior grupo de tipos de empresa.
# 1. Criar faixas de horário de 3 em 3 horas com transações efetuadas nesse horário - por empresa.
# 2. Plotar histograma de faixa de horário de transação. 1 gráfico por CNAE (não empresa).
# 3. Verificar a possibilidade de clusterização utilizando KNN caso gráfico de um CNAE seja muito divergente 
#    (sem faixa de horários muito contrastantes), verificando se cabem clusters de clientes, dado horário de transações.

# Faixas de horário:
# 02 h as 05 h
# 05 h as 08 h
# 08 h as 11 h
# 11 h as 14 h
# 14 h as 17 h
# 17 h as 20 h
# 20 h as 23 h
# 23 h as 02 h

In [3]:
# importar arquivo como csv

import pandas as pd

combined_csv = pd.read_csv("../combined_csv.csv", skiprows = 1, names =["dominio","empresa","3","datahora","5","6","7","8"])

print(combined_csv.shape)

D:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(1499999, 8)


In [4]:
# removendo colunas desnecessarias

combined_csv.drop("3", inplace=True, axis=1)
combined_csv.drop("5", inplace=True, axis=1)
combined_csv.drop("6", inplace=True, axis=1)
combined_csv.drop("7", inplace=True, axis=1)
combined_csv.drop("8", inplace=True, axis=1)

In [5]:
# tratamento de data e hora

from datetime import datetime

data_e_hora_parsed = pd.to_datetime(combined_csv.loc[:, "datahora"], format="%Y-%m-%dT%H:%M:%S.000Z",errors="coerce")

combined_csv["datahora"] = data_e_hora_parsed

print(combined_csv.head(4))

                                             dominio  \
0  6139646137356433363237396637323937336161383832...   
1  6139646137356433363237396637323937336161383832...   
2  6139646137356433363237396637323937336161383832...   
3  6139646137356433363237396637323937336161383832...   

                        empresa            datahora  
0  21404ebaa0106ca20ae283c89271 2018-11-19 16:40:45  
1  510766392a95afd41c6bd526381a 2018-12-07 09:56:37  
2  21404ebaa0106ca20ae283c89271 2018-10-18 08:29:47  
3  8ef4d40b852168ea8e3bbc4c89a0 2018-10-15 09:18:55  


In [6]:
# limpeza dos campos NaT

df_datas_corretas = combined_csv[combined_csv["datahora"].notnull()] 
df_datas_incorretas = combined_csv[combined_csv["datahora"].isnull()]

print(df_datas_corretas.shape)
print(df_datas_corretas.head(4))

(1499980, 3)
                                             dominio  \
0  6139646137356433363237396637323937336161383832...   
1  6139646137356433363237396637323937336161383832...   
2  6139646137356433363237396637323937336161383832...   
3  6139646137356433363237396637323937336161383832...   

                        empresa            datahora  
0  21404ebaa0106ca20ae283c89271 2018-11-19 16:40:45  
1  510766392a95afd41c6bd526381a 2018-12-07 09:56:37  
2  21404ebaa0106ca20ae283c89271 2018-10-18 08:29:47  
3  8ef4d40b852168ea8e3bbc4c89a0 2018-10-15 09:18:55  


In [7]:
# transformando em horário

df_horarios_transacoes = df_datas_corretas["datahora"].dt.hour

df_datas_corretas["hora"] = df_horarios_transacoes

df_datas_corretas.drop("datahora", inplace=True, axis=1)

print(df_datas_corretas.shape)
print(df_datas_corretas.head(4))

(1499980, 3)
                                             dominio  \
0  6139646137356433363237396637323937336161383832...   
1  6139646137356433363237396637323937336161383832...   
2  6139646137356433363237396637323937336161383832...   
3  6139646137356433363237396637323937336161383832...   

                        empresa  hora  
0  21404ebaa0106ca20ae283c89271    16  
1  510766392a95afd41c6bd526381a     9  
2  21404ebaa0106ca20ae283c89271     8  
3  8ef4d40b852168ea8e3bbc4c89a0     9  


D:\Anaconda\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
D:\Anaconda\lib\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [19]:
# Agrupando as empresas e criando um dataframe de horário por cada transação

empresas = df_datas_corretas.empresa.unique()

COLUNAS = [
    '2 a 5h',
    '5 a 8h',
    '8 a 11h',
    '11 a 14h',
    '14 a 17h',
    '17 a 20h',
    '20 a 23h',
    '23 a 2h',
    'Total'
]

frequencia_horario_empresas = pd.DataFrame(columns=COLUNAS,index=empresas)
for col in frequencia_horario_empresas.columns:
    frequencia_horario_empresas[col].values[:] = 0
    
print(frequencia_horario_empresas.head(3))

                             2 a 5h 5 a 8h 8 a 11h 11 a 14h 14 a 17h 17 a 20h  \
21404ebaa0106ca20ae283c89271      0      0       0        0        0        0   
510766392a95afd41c6bd526381a      0      0       0        0        0        0   
8ef4d40b852168ea8e3bbc4c89a0      0      0       0        0        0        0   

                             20 a 23h 23 a 2h Total  
21404ebaa0106ca20ae283c89271        0       0     0  
510766392a95afd41c6bd526381a        0       0     0  
8ef4d40b852168ea8e3bbc4c89a0        0       0     0  


In [20]:
# populando o dataframe de frequencias de horario por empresa

for index, row in df_datas_corretas.iterrows():
    if index % 100000 == 0:
        print(index)
    if row["hora"] < 2:
        frequencia_horario_empresas.loc[row["empresa"], "23 a 2h"] += 1
    elif row["hora"] < 5:
        frequencia_horario_empresas.loc[row["empresa"], "2 a 5h"] += 1
    elif row["hora"] < 8:
        frequencia_horario_empresas.loc[row["empresa"], "5 a 8h"] += 1
    elif row["hora"] < 11:
        frequencia_horario_empresas.loc[row["empresa"], "8 a 11h"] += 1
    elif row["hora"] < 14:
        frequencia_horario_empresas.loc[row["empresa"], "11 a 14h"] += 1
    elif row["hora"] < 17:
        frequencia_horario_empresas.loc[row["empresa"], "14 a 17h"] += 1
    elif row["hora"] < 20:
        frequencia_horario_empresas.loc[row["empresa"], "17 a 20h"] += 1
    elif row["hora"] < 23:
        frequencia_horario_empresas.loc[row["empresa"], "20 a 23h"] += 1
    else:
        frequencia_horario_empresas.loc[row["empresa"], "23 a 2h"] += 1
        
    frequencia_horario_empresas.loc[row["empresa"], "Total"] += 1
        

0
100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000


In [22]:
# Checando se resultado bate com valor total

print(frequencia_horario_empresas.head(5))

total = 0

for index, row in frequencia_horario_empresas.iterrows():
    total += row["Total"]
    
print(total)

                              2 a 5h  5 a 8h  8 a 11h  11 a 14h  14 a 17h  \
21404ebaa0106ca20ae283c89271     114     589     1225      2093      1765   
510766392a95afd41c6bd526381a      25     385      809       855       846   
8ef4d40b852168ea8e3bbc4c89a0     289     575      837       809       884   
617b9dcd2479b7e85da7a90b56c4      53     202      426       482       405   
5dc07a1ef6f3839d858729107f26       0     174      317       301       304   

                              17 a 20h  20 a 23h  23 a 2h  Total  
21404ebaa0106ca20ae283c89271      2759      1755      798  11098  
510766392a95afd41c6bd526381a       893       769      179   4761  
8ef4d40b852168ea8e3bbc4c89a0       977       841      524   5736  
617b9dcd2479b7e85da7a90b56c4       480       331      108   2487  
5dc07a1ef6f3839d858729107f26       387       135        0   1618  
1499980


In [27]:
# Fazendo o cálculo de % de transações por faixa de horário

porcentagem_frequencia_empresas = frequencia_horario_empresas.copy()

for index, row in frequencia_horario_empresas.iterrows():
    porcentagem_frequencia_empresas.loc[index,["2 a 5h"]]   = row["2 a 5h"]   / row["Total"]
    porcentagem_frequencia_empresas.loc[index,["5 a 8h"]]   = row["5 a 8h"]   / row["Total"]
    porcentagem_frequencia_empresas.loc[index,["8 a 11h"]]  = row["8 a 11h"]  / row["Total"]
    porcentagem_frequencia_empresas.loc[index,["11 a 14h"]] = row["11 a 14h"] / row["Total"]
    porcentagem_frequencia_empresas.loc[index,["14 a 17h"]] = row["14 a 17h"] / row["Total"]
    porcentagem_frequencia_empresas.loc[index,["17 a 20h"]] = row["17 a 20h"] / row["Total"]
    porcentagem_frequencia_empresas.loc[index,["20 a 23h"]] = row["20 a 23h"] / row["Total"]
    porcentagem_frequencia_empresas.loc[index,["23 a 2h"]]  = row["23 a 2h"]  / row["Total"]

print(frequencia_horario_empresas.head(5))
print(porcentagem_frequencia_empresas.head(5))

                              2 a 5h  5 a 8h  8 a 11h  11 a 14h  14 a 17h  \
21404ebaa0106ca20ae283c89271     114     589     1225      2093      1765   
510766392a95afd41c6bd526381a      25     385      809       855       846   
8ef4d40b852168ea8e3bbc4c89a0     289     575      837       809       884   
617b9dcd2479b7e85da7a90b56c4      53     202      426       482       405   
5dc07a1ef6f3839d858729107f26       0     174      317       301       304   

                              17 a 20h  20 a 23h  23 a 2h  Total  
21404ebaa0106ca20ae283c89271      2759      1755      798  11098  
510766392a95afd41c6bd526381a       893       769      179   4761  
8ef4d40b852168ea8e3bbc4c89a0       977       841      524   5736  
617b9dcd2479b7e85da7a90b56c4       480       331      108   2487  
5dc07a1ef6f3839d858729107f26       387       135        0   1618  
                                2 a 5h    5 a 8h   8 a 11h  11 a 14h  \
21404ebaa0106ca20ae283c89271  0.010272  0.053073  0.110380  0.1

In [28]:
# Exportando arquivo de porcentagens

export_csv = porcentagem_frequencia_empresas.to_csv (r'frequencia_transacoes_por_empresa.csv', index_label = "empresa")

print(porcentagem_frequencia_empresas.shape)

(1548, 9)
